<center><b><font size=6>Supervised learning – classification<b><center>

This notebook implements the supervised learning component of the SSH Shell Attack session project. The goal is to classify attack session tactics based on the provided dataset.

1. **Data Splitting**

2. **Baseline Model Implementation**

3. **Hyperparameter Tuning**
    
4. **Feature Experimentation**
    
5. **Result Analysis**

<center><b><font size=5>Data Splitting<b><center>

Split the dataset into training and testing sets.

**Steps:**

- Load the dataset from the given `.parquet` file.
- Explore basic statistics and check for missing data.
- Visualize the distribution of intents in the dataset.

### Load the dataset

- text
- text

### Check for ... / Make some actions

- text
- text

### Split the dataset

- text
- text

<center><b><font size=5>Baseline Model Implementation<b><center>

Train at least two ML models and evaluate their performance.

**Steps:**

- Train two different ML models (e.g., Logistic Regression and Random Forest).
- Evaluate the models on the training and testing sets.
- Generate confusion matrices and classification reports for each model.

### Chose models and train

- text
- text

### Evaluate

- text
- text

### Confusion matrices / Classification reports

- text
- text

<center><b><font size=5>Hyperparameter Tuning<b><center>

Fine-tune the hyperparameters of the ML models using cross-validation.

**Steps:**

TO BE CHECKED

- Use GridSearchCV or RandomizedSearchCV to tune hyperparameters.
- Evaluate the performance of the tuned models.
- Compare the results with baseline performance.

### text

- text
- text

### text

- text
- text

### text

- text
- text

<center><b><font size=5>Feature Experimentation<b><center>

Explore the impact of different features on model performance.

**Steps:**

- Experiment with feature combinations (e.g., adding temporal features).
- Observe performance changes with each combination.
- Document successful and unsuccessful experiments.

### text

- text
- text

### text

- text
- text

### text

- text
- text

<center><b><font size=5>Result Analysis<b><center>

Analyze and summarize the results for each intent.

**Steps:**

